In [ ]:
from map import Map, Marker
import db

In [ ]:
locations = db.query(
    """
select round(lat, 5) as lat, round(lon, 6) as lon, count(*) as num_events

from chicago_escooter_events

group by 1,2
having num_events > 10
order by 3 desc
"""
)
locations

In [ ]:
m = Map((41.85, -87.5))
for row in locations.fetchall():
    m.markers.append(Marker((row[0], row[1]), f"Num Events: {row[2]}", "gear"))
m

In [ ]:
db.query(
    """
select vendor, max(start_time) as start_time, max(end_time) as end_time, count(*) num_trips, sum(trip_distance) total_trip_distance

from raw.chicago_escooter_trips

group by 1
order by num_trips desc
"""
)

In [ ]:
db.query(
    """
    select 
    community_area_name, count(*) as num_events
    from chicago_escooter_events
    group by 1
    having num_events > 10
    order by 2 desc
"""
).df().plot.bar(
    x="community_area_name", title="Monthly Station Events", rot=45, figsize=(20, 5)
)

In [ ]:
db.query(
    """
 select 
    strftime(event_at, '%Y-%m') as month
    , count(*) as num_events
    from chicago_escooter_events e
    group by 1
    order by month asc
"""
).df().plot.bar(x="month", title="Monthly Events", rot=45)

In [ ]:
db.query(
    """
select * from raw.chicago_escooter_trips
"""
)